# Weekly Execution Progress

Exact outstanding-order tracking from `{TEAM_ID}/targets.csv`.

Key metric:
- `% outstanding = abs(weekly_target_w - actual_w) / abs(weekly_target_w - start_w)`

This notebook plots both portfolio-level and symbol-level progress.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

from QuantConnect import *
from QuantConnect.Research import QuantBook
from config import TEAM_ID

qb = QuantBook()
print('QuantBook initialized')


def read_csv_from_store(key):
    try:
        if not qb.ObjectStore.ContainsKey(key):
            print(f'ObjectStore key not found: {key}')
            return None
        content = qb.ObjectStore.Read(key)
        if not content:
            print(f'Empty ObjectStore key: {key}')
            return None
        return pd.read_csv(StringIO(content))
    except Exception as e:
        print(f'Error reading {key}: {e}')
        return None


## Data Loading — Targets Log

Loads `targets.csv` and computes the outstanding and filled percentage for each symbol-week-day row from the difference between weekly target and actual weights. The head preview confirms that `weekly_target_w`, `scheduled_fraction`, and `actual_w` columns are present and correctly typed. This DataFrame is the foundation for all portfolio-level and symbol-level progress charts below.

## Weekly Summary Table — Final Outstanding %

This table aggregates execution progress to one row per rebalance week, reporting the total order size and what fraction remained outstanding at the end of the final scaling day. Weeks with a high `final_outstanding_pct` flag execution failures where limit orders were not filled before the next rebalance. Use this table as a starting point to investigate specific weeks where scaling fell short.

## Portfolio Outstanding % — Daily and Day-In-Week Profiles

These two plots track how quickly the strategy fills its weekly orders. The top chart shows the raw daily outstanding percentage over time, making it easy to spot periods of chronic underfilling. The lower chart averages outstanding percentage by scaling-day index across all weeks, producing a decay curve that reveals the typical fill profile — a well-functioning strategy should show most outstanding declining by scale days 1 or 2.

## Symbol Order Size Ranking and Top-Symbol Progress Chart

This section first ranks all symbols by average weekly order size, identifying which names generate the most trading activity over the backtest. The chart then plots the outstanding percentage and scheduled fraction for the most actively traded symbol, providing a granular check of whether that specific name tracks the scheduled scaling curve. A persistent gap between the scheduled and outstanding lines for a given symbol suggests its limit offsets are too aggressive for its typical liquidity.

In [ ]:
df_targets = read_csv_from_store(f'{TEAM_ID}/targets.csv')
df_snapshots = read_csv_from_store(f'{TEAM_ID}/daily_snapshots.csv')
df_signals = read_csv_from_store(f'{TEAM_ID}/signals.csv')

if df_targets is None:
    raise ValueError('targets.csv is required. Run a backtest after enabling target-state logging.')

df_targets['date'] = pd.to_datetime(df_targets['date'])
numeric_cols = ['start_w', 'weekly_target_w', 'scheduled_fraction', 'scheduled_w', 'actual_w', 'scale_day']
for col in numeric_cols:
    df_targets[col] = pd.to_numeric(df_targets[col], errors='coerce').fillna(0.0)

if df_snapshots is not None:
    df_snapshots['date'] = pd.to_datetime(df_snapshots['date'])

if df_signals is not None:
    df_signals['date'] = pd.to_datetime(df_signals['date'])
    df_signals['week_id'] = df_signals['date'].dt.strftime('%Y-%m-%d')
    df_signals['magnitude'] = pd.to_numeric(df_signals['magnitude'], errors='coerce').fillna(0.0).abs()
    week_signal = (
        df_signals[['week_id', 'symbol', 'magnitude']]
        .drop_duplicates(['week_id', 'symbol'], keep='last')
    )
else:
    week_signal = pd.DataFrame(columns=['week_id', 'symbol', 'magnitude'])

print(f'target rows: {len(df_targets):,}')
print(f'week-signal rows: {len(week_signal):,}')
display(df_targets.head())


In [ ]:
df = df_targets.copy()
df['week_id'] = df['week_id'].astype(str)
df['scale_day'] = pd.to_numeric(df['scale_day'], errors='coerce')
df = df.merge(week_signal, on=['week_id', 'symbol'], how='left')

def tier_from_mag(m):
    if pd.isna(m):
        return 'unknown'
    if m >= 0.7:
        return 'strong'
    if m >= 0.3:
        return 'moderate'
    return 'weak'

df['signal_tier_raw'] = df['magnitude'].apply(tier_from_mag)

df['total_week_order_abs'] = (df['weekly_target_w'] - df['start_w']).abs()
df['remaining_order_abs'] = (df['weekly_target_w'] - df['actual_w']).abs()
df['remaining_order_abs'] = np.minimum(df['remaining_order_abs'], df['total_week_order_abs'])

df['outstanding_pct'] = np.where(
    df['total_week_order_abs'] > 1e-10,
    df['remaining_order_abs'] / df['total_week_order_abs'],
    0.0
)
df['filled_pct'] = 1.0 - df['outstanding_pct']

# Build week-level tier map. Missing signal often means an exit week.
base_tiers = ['strong', 'moderate', 'weak']
week_tier = (
    df[['week_id', 'symbol', 'signal_tier_raw', 'start_w', 'weekly_target_w', 'total_week_order_abs']]
      .drop_duplicates(['week_id', 'symbol'])
      .sort_values(['symbol', 'week_id'])
      .copy()
)

eps = 1e-8
week_tier['is_exit_week'] = (
    week_tier['weekly_target_w'].abs() <= eps
) & (
    week_tier['start_w'].abs() > eps
)

week_tier['known_tier'] = week_tier['signal_tier_raw'].where(week_tier['signal_tier_raw'].isin(base_tiers))
week_tier['prev_known_tier'] = (
    week_tier.groupby('symbol')['known_tier']
             .transform(lambda s: s.ffill().shift(1))
)
week_tier['signal_tier'] = week_tier['signal_tier_raw']

# For unknown exit weeks, attribute to prior known tier when available.
mask_unknown_exit_with_parent = (
    (week_tier['signal_tier'] == 'unknown') &
    week_tier['is_exit_week'] &
    week_tier['prev_known_tier'].notna()
)
week_tier.loc[mask_unknown_exit_with_parent, 'signal_tier'] = week_tier.loc[mask_unknown_exit_with_parent, 'prev_known_tier']

# Remaining unknown exits are labeled explicitly as exit.
mask_unknown_exit = (week_tier['signal_tier'] == 'unknown') & week_tier['is_exit_week']
week_tier.loc[mask_unknown_exit, 'signal_tier'] = 'exit'

df = df.drop(columns=['signal_tier_raw'], errors='ignore')
df = df.merge(
    week_tier[['week_id', 'symbol', 'signal_tier_raw', 'signal_tier', 'is_exit_week']],
    on=['week_id', 'symbol'],
    how='left'
)

# Keep only rows with real weekly orders and valid 5-day scaling index.
df = df[df['total_week_order_abs'] > 1e-10].copy()
df['day_in_week'] = df['scale_day'].round().astype('Int64')
df = df[df['day_in_week'].between(0, 4)].copy()

# Keep complete 5-day symbol-weeks only (days 0..4 all present).
coverage = (
    df.groupby(['week_id', 'symbol'])['day_in_week']
      .agg(days='nunique', min_day='min', max_day='max')
      .reset_index()
)
complete_pairs = coverage[
    (coverage['days'] == 5) &
    (coverage['min_day'] == 0) &
    (coverage['max_day'] == 4)
][['week_id', 'symbol']]

df_complete = df.merge(complete_pairs, on=['week_id', 'symbol'], how='inner')
df_complete = df_complete.sort_values(['week_id', 'symbol', 'day_in_week', 'date'])

portfolio = (
    df_complete.groupby(['date', 'week_id', 'day_in_week'], as_index=False)
      .agg(total_week_order_abs=('total_week_order_abs', 'sum'),
           remaining_order_abs=('remaining_order_abs', 'sum'))
      .sort_values('date')
)
portfolio['outstanding_pct'] = np.where(
    portfolio['total_week_order_abs'] > 1e-10,
    portfolio['remaining_order_abs'] / portfolio['total_week_order_abs'],
    0.0
)
portfolio['filled_pct'] = 1.0 - portfolio['outstanding_pct']

summary = (
    portfolio.groupby('week_id', as_index=False)
             .agg(week_total_order=('total_week_order_abs', 'sum'),
                  final_outstanding_pct=('outstanding_pct', 'last'))
             .sort_values('week_id')
)

tier_coverage = (
    week_tier.groupby('signal_tier', as_index=False)
             .agg(
                 symbol_weeks=('week_id', 'size'),
                 total_order_abs=('total_week_order_abs', 'sum')
             )
             .sort_values('total_order_abs', ascending=False)
)
total_order = tier_coverage['total_order_abs'].sum()
tier_coverage['order_share_pct'] = np.where(
    total_order > 1e-10,
    100 * tier_coverage['total_order_abs'] / total_order,
    np.nan
)

print(f'Complete 5-day symbol-weeks: {len(complete_pairs):,} of {len(coverage):,}')
display(tier_coverage)
display(summary.tail(10))


In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(portfolio['date'], 100 * portfolio['outstanding_pct'], color='#1f77b4', linewidth=1.8)
ax.set_title('Portfolio Outstanding Weekly Order % (Daily, Complete Weeks)')
ax.set_ylabel('% outstanding')
ax.set_xlabel('Date')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

day_profile = (
    portfolio.groupby('day_in_week', as_index=False)['outstanding_pct']
             .agg(['mean', 'median'])
             .reset_index()
)
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(day_profile['day_in_week'], 100 * day_profile['mean'], marker='o', label='Mean')
ax.plot(day_profile['day_in_week'], 100 * day_profile['median'], marker='s', label='Median')
ax.set_title('Outstanding % by Day-In-Week (Complete Weeks)')
ax.set_ylabel('% outstanding')
ax.set_xlabel('Scaling day index')
ax.set_xticks([0, 1, 2, 3, 4])
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

tier_order = ['strong', 'moderate', 'weak', 'exit']
tier_day_profile = (
    df_complete[df_complete['signal_tier'].isin(tier_order)]
      .groupby(['signal_tier', 'day_in_week'], as_index=False)
      .agg(
          total_order_abs=('total_week_order_abs', 'sum'),
          remaining_order_abs=('remaining_order_abs', 'sum'),
          mean_outstanding_pct=('outstanding_pct', 'mean'),
          median_outstanding_pct=('outstanding_pct', 'median'),
          observations=('outstanding_pct', 'count')
      )
)
tier_day_profile['weighted_outstanding_pct'] = np.where(
    tier_day_profile['total_order_abs'] > 1e-10,
    tier_day_profile['remaining_order_abs'] / tier_day_profile['total_order_abs'],
    np.nan
)
tier_day_profile['signal_tier'] = pd.Categorical(
    tier_day_profile['signal_tier'],
    categories=tier_order,
    ordered=True
)
tier_day_profile = tier_day_profile.sort_values(['signal_tier', 'day_in_week'])

palette = {
    'strong': '#2ca02c',
    'moderate': '#1f77b4',
    'weak': '#ff7f0e',
    'exit': '#7f7f7f'
}
linestyle_map = {
    'strong': '-',
    'moderate': '-',
    'weak': '-',
    'exit': '--'
}

fig, ax = plt.subplots(figsize=(12, 5))
for tier in tier_order:
    sub = tier_day_profile[tier_day_profile['signal_tier'] == tier]
    if sub.empty:
        continue
    ax.plot(
        sub['day_in_week'],
        100 * sub['weighted_outstanding_pct'],
        marker='o',
        linewidth=1.8,
        linestyle=linestyle_map[tier],
        color=palette[tier],
        label=tier.title()
    )

ax.set_title('Outstanding % by Day-In-Week by Signal Tier (Complete Weeks)')
ax.set_ylabel('% outstanding')
ax.set_xlabel('Scaling day index')
ax.set_xticks([0, 1, 2, 3, 4])
ax.grid(alpha=0.3)
ax.legend(title='Signal tier')
plt.tight_layout()
plt.show()

display(tier_day_profile)


In [ ]:
symbol_order = (
    df.groupby('symbol', as_index=False)['total_week_order_abs']
      .mean()
      .sort_values('total_week_order_abs', ascending=False)
)
display(symbol_order.head(15))

selected_symbol = symbol_order['symbol'].iloc[0] if len(symbol_order) else None
if selected_symbol is not None:
    dfx = df[df['symbol'] == selected_symbol].copy().sort_values('date')
    fig, ax = plt.subplots(figsize=(14, 6))
    ax.plot(dfx['date'], 100 * dfx['outstanding_pct'], label='Outstanding %', color='#d62728')
    ax.plot(dfx['date'], 100 * dfx['scheduled_fraction'], label='Scheduled % of target', color='#2ca02c', alpha=0.8)
    ax.set_title(f'{selected_symbol} Weekly Execution Progress')
    ax.set_ylabel('%')
    ax.set_xlabel('Date')
    ax.legend()
    ax.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
